# Imports and Constants

In [91]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import h3
import folium
import osmnx as ox
from shapely import wkt
from folium.plugins import HeatMap
from shapely.geometry import Polygon
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.geometry import MultiPolygon
import os
from area import area
from shapely.geometry import mapping
from pyproj import Geod

In [92]:
# moscow' name as a constant
MOSCOW = "Moscow"
# the main resolution used across the notebook
RESOLUTION = 9
# a constant used to scale the areas
KM_2 = 10 ** 3

## General functions

In [93]:
def_cols = ['h3', 'lat', 'lng', 'area']

In [94]:
# get the h3 address of the hexagone for which the geometry belongs to
def get_h3_geo(g, res=RESOLUTION):
    # longitude
    lng = g.x if isinstance(g, Point) else g.centroid.x     
    # latitude 
    lat = g.y if isinstance(g, Point) else g.centroid.y
    # get h3
    h3_addr = h3.geo_to_h3(lat=lat, lng=lng, resolution=res)
    return h3_addr, lat, lng

# this function returns the area of the passed geometry divided by the scaling constant
def get_area(geo):
    geod = Geod(ellps="WGS84") 
    return abs(geod.geometry_area_perimeter(geo)[0]) / KM_2

In [95]:
# this function extracts the h3 address of each geometry in each row of a pandas dataframe
def get_h3(row, res=RESOLUTION):
    g = row['geometry']
    row['h3'], row['lat'], row['lng'] = get_h3_geo(g, res)
    return row

def get_default(row):
    row_geo = get_h3(row)
    row_geo['area'] = get_area(row['geometry'])
    return row_geo

In [96]:
def osm_query(city, tag):
    """extracts all elements that satisfy the tag passed as an argument and returns them in a geopandas

    Args:
        city (_type_): string describing the city in question
        tag (_type_): OSM tags to filter elements

    Returns:

    """
    gdf = ox.geometries_from_place(city, tag).reset_index()
    print(gdf.shape)
    return gdf

In [97]:
def get_h3_point(point):
    # return the h3 address of the hexagone for which the point belongs to
    h3_a, lat, lng = get_h3_geo(point)
    return [h3_a]    

def get_h3_lineString(ls: LineString):
    """this function considers all the points belonging to LineString object and returns a set of all h3 addresses of hexagones
    intersecting this geometry

    Args:
        ls (LineString): A lineString object representing a OpenStreetMap object, generally: highway, road, street...

    Returns:
        set: the set of all h3_address of the hexagones that are spanned by the LineString object
    """
    
    h3_s = set()
    x, y = ls.coords.xy
    for p_x, p_y in zip(x, y):
        # получить адрес h3 шестиугольника, к которому принадлежит эта конкретная точка
        
        h3_addr = h3.geo_to_h3(lat=p_y, lng=p_x, resolution=RESOLUTION)
        
        # добавьте адрес в набор
        h3_s.add(h3_addr)
    return h3_s

def get_h3_Poly(poly: Polygon):
    """this function considers all the points belonging to Polygon object and returns a set of all h3 addresses of hexagones
    intersecting this geometry

    Args:
        poly (Polygone): A Polygone object representing a OpenStreetMap element: large enough not be represented as a point

    Returns:
        set: the set of all h3_address of the hexagones that are spanned by the Polygon object
    """
    h3_s = set()
    x, y = poly.exterior.coords.xy
    for p_x,p_y in zip(x, y):
        # get the h3 address of the hexagone to which this particular point belongs
        h3_addr = h3.geo_to_h3(lat=p_y, lng=p_x, resolution=RESOLUTION)
        # add the address to the set
        h3_s.add(h3_addr)
    return h3_s


def get_h3_MultiPoly(m_poly: MultiPolygon):
    h3_s = set()
    for poly in m_poly.geoms:
        h3_s.update(get_h3_Poly(poly))
    return h3_s

def get_h3_area(g):
    """This function gets the area as well as the set of h3 addresses of hexagones spanned by the given geometry"""
    
    # first get the area
    area = get_area(g)
    addresses = None

    # call the corresponding function depending on the geometry's data type
        
    if isinstance(g, Point):
        addresses = get_h3_point(g)
    elif isinstance(g, LineString):
        addresses = get_h3_lineString(g)
    elif isinstance(g, Polygon):
        addresses = get_h3_Poly(g)
    else:
        addresses = get_h3_MultiPoly(g)
    return area, addresses
        

In [98]:
COLS = ['h3', 'count', 'area']
def get_count_area_tag(tag):
    
    # extract the data from osm
    g = osm_query(MOSCOW, tag).loc[:, 'geometry']
        
    
    # h3_count: maps each h3 address to the number of elements in geopandas "g" that intersect the hexagone h3
    # h3_area: maos each h3 address to the sum of areas of elements in geopandas "g" that intersect the hexagone h3
    h3_count = {}
    h3_area = {}
                
    for geo in g:
        # get the area and the unique address associated with this element: geometry
        area, adds = get_h3_area(geo)

        # for each address
        for ad in adds:
            # if this address was encoutered before increment its count and add the area of the element to its h3_area
            if ad in h3_count:
                h3_count[ad] += 1
                h3_area[ad] += area
            else:
                # if this address if first encountered then set its count to 1 and its area to the element's
                h3_count[ad] = 1
                h3_area[ad] = area

    # convert the dictionaries into dataframe                
    res = pd.DataFrame({"h3": list(h3_area.keys()), "count": list(h3_count.values()), "area": list(h3_area.values())})
    return res

In [99]:
def get_tags(tags, name):
    df_res = pd.DataFrame(data=[], columns=COLS)
    # for each specific tag
    for t in tags:
        # create the count and area dataframe associated with the specific dataframe
        df_tag = get_count_area_tag(t)
        # add it to previous dataframes    
        df_res = pd.concat([df_res, df_tag])
    
    # as several h3 address can repeat accross different tags: we group by h3 addresses
    # and sum up all their associated areas and counts
    
    df_res_area = pd.pivot_table(df_res, index='h3', values=['area'], aggfunc=[np.sum])
    df_res_count = pd.pivot_table(df_res, index='h3', values=['count'], aggfunc=[np.sum])
    final_df = pd.merge(df_res_count, df_res_area, right_index=True, left_index=True)
    final_df.columns = [f"{name}_count", f"{name}_area"]
    final_df.to_excel(os.path.join("osm_features", f"{name}.xlsx"))
    return final_df

## ОСОБЕННОСТИ ОБРАЗОВАНИЯ

In [100]:
tags_education = [{"amenity": "college"}, {"amenity": "school"}, {"amenity": "university"}]

In [101]:
edu_df = get_tags(tags_education, "education")

(281, 88)
(1949, 152)
(380, 194)


## ОСОБЕННОСТИ парковки

In [102]:
tags_parking = [{"amenity":"parking"}]
trans_df = get_tags(tags_parking, "parking")
# gdf_trans = get_final_df(def_cols, tags_parking, get_default, "transport")

(15069, 139)


In [103]:
tags_bus = [{"highway":"bus_stop"}]
bus_df = get_tags(tags_bus, "bus")
bus_df = bus_df.loc[:, ['bus_count']]

(9539, 115)


## Финансовые особенности

In [104]:
tags_financial = [{"amenity": "atm"}, {"amenity": "bank"}, {"amenity": "bureau_de_change"}]
financial_df = get_tags(tags_financial, "financial")

(2144, 91)
(2047, 118)
(129, 39)


## Особенности размещения

In [105]:
acc_tags = [{'building' : 'apartments'}, {'building' : 'detached'}, 
        {'building' : 'dormitory'}, {'building' : 'hotel'}, 
        {'building' : 'house'}]

In [106]:
acc_df = get_tags(acc_tags, "accomodation")

(30283, 168)
(219, 30)
(325, 67)
(121, 108)
(3040, 71)


# Коммерческие функции

In [107]:
commercial_tags = [{'building' : 'commercial'}, {'building' : 'retail'}, {'building' : 'supermarket'}]
commercial_df = get_tags(commercial_tags, "commercial")

(1864, 179)
(2072, 205)
(19, 36)


# Здравоохранение

In [108]:
health_care_tags = [{"amenity":"hospital"}, {"amenity": "clinic"}]

In [109]:
# gdf_health_care = get_final_df(def_cols, health_care_tags, get_default, "health_care")
df_health_care = get_tags(health_care_tags, "health_care")

(264, 100)
(1361, 158)


# Развлечения

In [110]:
enter_tags = [{"amenity":"gambling"}, {"amenity":"nightclub"}, {"amenity": "cinema"}, {"amenity": "conference_centre"}
              , {"amenity": "community_centre"}]
gdf_entertainment = get_tags(enter_tags, "entertainment")

(0, 2)
(90, 43)
(128, 76)
(0, 2)
(323, 123)


# пропитание

In [111]:
sus_tags = [{"amenity":"bar"}, {"amenity":"cafe"}, {"amenity":"fast_food"}, {"amenity":"food_court"}, 
            {"amenity":"ice_cream"}, {"amenity":"pub"}, {"amenity":"restaurant"}]

sustenance_df = get_tags(sus_tags, "sustenance")

(674, 93)
(3743, 189)
(3222, 186)
(59, 43)
(381, 64)
(329, 81)
(2468, 163)


# Правительство

In [112]:
gover_tags = [{"building":"government"}]
gover_df = get_tags(gover_tags, "government")

(96, 61)


In [113]:
sport_tags = [{"building":"sports_hall"}, {"building":"stadium"}]
sports_df = get_tags(sport_tags,"sports")

(15, 33)
(11, 38)


# АВТОМОБИЛЬНЫЕ ДОРОГИ

In [114]:
highway_tags = [{"highway":"primary"}, {"highway":"secondary"}, {"highway":"tertiary"}, {"highway":"residential"}, {"highway":"pedestrian"}]

In [115]:
def get_highway(tag, res=RESOLUTION):
    res_gdf = gpd.GeoDataFrame(data=[], columns=['h3', 'length', 'count'])
    loc = os.path.join("osm_features")
    g = osm_query(MOSCOW, tag).loc[:, ['geometry']]
    g_length = g.to_crs(epsg=3763)

        
    h_type = list(tag.values())[0]
    name = f"highway_{h_type}"
    # f"length_{name}"
    idx = 0
    for w, w_l in zip(g['geometry'], g_length['geometry']):
        # the total length of the highway (possibly an approximation)
        l = w_l.length / KM_2
        
        points_highway = {}
        
        if isinstance(w, LineString):
            # get the coordinates of the points within the highway
            x, y = w.coords.xy
            for p_x, p_y in zip(x, y):
                # get the hexagone to where the point in the high belongs
                h3_add = h3.geo_to_h3(lat=p_y, lng=p_x, resolution=res)
                # if this hexagone was already considered, then add 1 to its value
                if h3_add in points_highway:
                    points_highway[h3_add] += 1
                # otherwise set it to 1: as it is the first point 
                else:
                    points_highway[h3_add] = 1
        else: 
            print(type(w))        
            break
        
        for key, value in points_highway.items():    
            # # add a row with the hexagons 'h3' address, the length of the highway passing through that hexagone
            # # and the number of points in the intersection between the hexagonal and the highway            
            res_gdf.at[idx, 'h3'] = key
            res_gdf.at[idx, 'length'] = l
            res_gdf.at[idx, 'count'] = value
            idx += 1

    # как и сейчас, каждая строка содержит адрес h3 точки, принадлежащей автомагистрали
    # и длина пути, который он соответствует
    # пришло время сгруппироваться по h3_addresses
    # и вычислите количество точек на пересечении между шоссе и зоной 
       
    res_gdf = pd.pivot_table(res_gdf, index='h3', values=['length', 'count'], aggfunc=[np.sum])
    res_gdf.columns = [f'count_{name}', f'length_{name}']
    res_gdf.to_excel(os.path.join(loc, f'{name}.xlsx'))
    return res_gdf

In [116]:
highway_dfs = [get_highway(t) for t in highway_tags]

(3732, 133)
(8344, 173)
(7545, 185)
(4983, 174)
(631, 126)
<class 'shapely.geometry.polygon.Polygon'>


# Merging

In [117]:
hex_loc = os.path.join(f"data_{str(RESOLUTION)}", "regions_bus_traff.xlsx")
hex = pd.read_excel(hex_loc).set_index('h3')
print(hex.head())

                       lat        lng  bus_freq_count
h3                                                   
891181b0093ffff  55.442051  37.531711               0
891181b0097ffff  55.444627  37.534808               0
891181b009bffff  55.439178  37.533722               0
891181b0403ffff  55.440369  37.513295               0
891181b0407ffff  55.442944  37.516390               0


In [118]:
features_dfs = [edu_df, trans_df, bus_df, financial_df,
                acc_df, commercial_df
                , df_health_care, gdf_entertainment,
                sustenance_df, gover_df, sports_df]
features_dfs.extend(highway_dfs)

def merge_zone_feature(hex, df_feat):
    try:
        hex = hex.set_index('h3')
    except KeyError:
        pass
    try:
        df_feat = df_feat.set_index('h3')
    except:
        pass
    hex = pd.merge(hex, df_feat, how='left', right_index=True, left_index=True)
    return hex, df_feat
df = hex.copy() 
for feat in features_dfs:
    df, _ = merge_zone_feature(df, feat)


In [119]:
df = df.fillna(0)
df.to_excel(os.path.join("osm_features", f"training_data_{str(RESOLUTION)}.xlsx"))